# 02 - 技术指标调试 (Indicator Debug Lab)

本 Notebook 用于交互式调试和可视化技术指标。

内容包括：
1. 指标计算与可视化
2. 参数调整与对比
3. 信号生成与回测
4. 多指标组合分析

## 1. 环境准备

In [ ]:
# 导入必要的库
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from datetime import datetime, timedelta

# 设置中文显示
matplotlib.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei']
matplotlib.rcParams['axes.unicode_minus'] = False

# 设置图表样式
plt.style.use('seaborn-v0_8-whitegrid')

# 掘金 SDK
from gm.api import *

# 添加 test 目录到路径 (导入自定义指标)
sys.path.insert(0, '../test')

print("Environment ready!")

In [ ]:
# 设置 Token
TOKEN = 'your_token_here'
set_token(TOKEN)

print("Token set successfully!")

In [ ]:
# 获取测试数据
symbol = 'SHSE.600000'
start_date = '2023-01-01'
end_date = '2023-12-31'

df = history(
    symbol=symbol,
    frequency='1d',
    start_time=start_date,
    end_time=end_date,
    fields='open,high,low,close,volume',
    adjust=ADJUST_PREV,
    df=True
)

print(f"获取 {symbol} 数据: {len(df)} 条")
df.head()

## 2. 指标计算函数库

In [ ]:
# ==============================================================================
# 技术指标计算函数
# ==============================================================================

def sma(series, period):
    """简单移动平均"""
    return series.rolling(window=period).mean()

def ema(series, period):
    """指数移动平均"""
    return series.ewm(span=period, adjust=False).mean()

def macd(close, fast=12, slow=26, signal=9):
    """MACD 指标"""
    exp1 = ema(close, fast)
    exp2 = ema(close, slow)
    dif = exp1 - exp2
    dea = ema(dif, signal)
    histogram = 2 * (dif - dea)
    return dif, dea, histogram

def rsi(close, period=14):
    """RSI 指标"""
    delta = close.diff()
    gain = delta.where(delta > 0, 0)
    loss = (-delta).where(delta < 0, 0)
    avg_gain = gain.ewm(span=period, adjust=False).mean()
    avg_loss = loss.ewm(span=period, adjust=False).mean()
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

def bollinger_bands(close, period=20, std_dev=2):
    """布林带"""
    middle = sma(close, period)
    std = close.rolling(window=period).std()
    upper = middle + std_dev * std
    lower = middle - std_dev * std
    return upper, middle, lower

def kdj(high, low, close, n=9, m1=3, m2=3):
    """KDJ 指标"""
    llv = low.rolling(window=n).min()
    hhv = high.rolling(window=n).max()
    rsv = (close - llv) / (hhv - llv) * 100
    k = rsv.ewm(span=m1, adjust=False).mean()
    d = k.ewm(span=m2, adjust=False).mean()
    j = 3 * k - 2 * d
    return k, d, j

def atr(high, low, close, period=14):
    """ATR 真实波幅"""
    tr1 = high - low
    tr2 = abs(high - close.shift(1))
    tr3 = abs(low - close.shift(1))
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    return tr.rolling(window=period).mean()

print("指标函数已加载")

## 3. 移动平均线可视化

In [ ]:
# 计算不同周期的移动平均
df['MA5'] = sma(df['close'], 5)
df['MA10'] = sma(df['close'], 10)
df['MA20'] = sma(df['close'], 20)
df['MA60'] = sma(df['close'], 60)
df['EMA20'] = ema(df['close'], 20)

# 绘图
fig, ax = plt.subplots(figsize=(14, 7))

ax.plot(df.index, df['close'], label='收盘价', color='black', linewidth=1.5)
ax.plot(df.index, df['MA5'], label='MA5', color='red', linewidth=1, alpha=0.8)
ax.plot(df.index, df['MA10'], label='MA10', color='orange', linewidth=1, alpha=0.8)
ax.plot(df.index, df['MA20'], label='MA20', color='blue', linewidth=1, alpha=0.8)
ax.plot(df.index, df['MA60'], label='MA60', color='purple', linewidth=1, alpha=0.8)
ax.plot(df.index, df['EMA20'], label='EMA20', color='green', linestyle='--', linewidth=1)

ax.set_ylabel('价格', fontsize=12)
ax.set_xlabel('日期', fontsize=12)
ax.set_title(f'{symbol} 移动平均线对比', fontsize=14)
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. MACD 指标分析

In [ ]:
# 计算 MACD
df['DIF'], df['DEA'], df['MACD'] = macd(df['close'], fast=12, slow=26, signal=9)

# 绘制 MACD 图
fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True,
                         gridspec_kw={'height_ratios': [2, 1]})

# 价格图
ax1 = axes[0]
ax1.plot(df.index, df['close'], label='收盘价', color='black', linewidth=1.5)
ax1.set_ylabel('价格', fontsize=12)
ax1.set_title(f'{symbol} MACD 分析', fontsize=14)
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# MACD 图
ax2 = axes[1]
ax2.plot(df.index, df['DIF'], label='DIF', color='blue', linewidth=1)
ax2.plot(df.index, df['DEA'], label='DEA', color='orange', linewidth=1)

# MACD 柱状图
colors = ['red' if x >= 0 else 'green' for x in df['MACD']]
ax2.bar(df.index, df['MACD'], color=colors, alpha=0.6, width=1)
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)

ax2.set_ylabel('MACD', fontsize=12)
ax2.set_xlabel('日期', fontsize=12)
ax2.legend(loc='upper left')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# MACD 参数对比
params_list = [
    (12, 26, 9, 'blue'),    # 标准参数
    (8, 17, 9, 'red'),      # 快速参数
    (19, 39, 9, 'green'),   # 慢速参数
]

fig, axes = plt.subplots(len(params_list), 1, figsize=(14, 3*len(params_list)), sharex=True)

for i, (fast, slow, sig, color) in enumerate(params_list):
    dif, dea, histogram = macd(df['close'], fast, slow, sig)
    
    ax = axes[i]
    ax.plot(df.index, dif, label='DIF', color='blue', linewidth=1)
    ax.plot(df.index, dea, label='DEA', color='orange', linewidth=1)
    bar_colors = ['red' if x >= 0 else 'green' for x in histogram]
    ax.bar(df.index, histogram, color=bar_colors, alpha=0.6, width=1)
    ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax.set_title(f'MACD({fast},{slow},{sig})', fontsize=12)
    ax.legend(loc='upper left')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. RSI 指标分析

In [ ]:
# 计算不同周期的 RSI
df['RSI6'] = rsi(df['close'], 6)
df['RSI14'] = rsi(df['close'], 14)
df['RSI24'] = rsi(df['close'], 24)

# 绘制 RSI 图
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True,
                         gridspec_kw={'height_ratios': [2, 1]})

# 价格图
ax1 = axes[0]
ax1.plot(df.index, df['close'], label='收盘价', color='black', linewidth=1.5)
ax1.set_ylabel('价格', fontsize=12)
ax1.set_title(f'{symbol} RSI 分析', fontsize=14)
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# RSI 图
ax2 = axes[1]
ax2.plot(df.index, df['RSI6'], label='RSI(6)', color='red', linewidth=1)
ax2.plot(df.index, df['RSI14'], label='RSI(14)', color='blue', linewidth=1)
ax2.plot(df.index, df['RSI24'], label='RSI(24)', color='green', linewidth=1)

# 超买超卖线
ax2.axhline(y=70, color='red', linestyle='--', linewidth=1, alpha=0.7)
ax2.axhline(y=30, color='green', linestyle='--', linewidth=1, alpha=0.7)
ax2.axhline(y=50, color='gray', linestyle='-', linewidth=0.5)

ax2.fill_between(df.index, 70, 100, color='red', alpha=0.1)
ax2.fill_between(df.index, 0, 30, color='green', alpha=0.1)

ax2.set_ylim(0, 100)
ax2.set_ylabel('RSI', fontsize=12)
ax2.set_xlabel('日期', fontsize=12)
ax2.legend(loc='upper left')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. 布林带分析

In [ ]:
# 计算布林带
df['BB_upper'], df['BB_middle'], df['BB_lower'] = bollinger_bands(df['close'], 20, 2)

# 计算布林带宽度和 %B
df['BB_width'] = (df['BB_upper'] - df['BB_lower']) / df['BB_middle']
df['BB_pctB'] = (df['close'] - df['BB_lower']) / (df['BB_upper'] - df['BB_lower'])

# 绘制布林带图
fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=True,
                         gridspec_kw={'height_ratios': [3, 1, 1]})

# 价格和布林带
ax1 = axes[0]
ax1.plot(df.index, df['close'], label='收盘价', color='black', linewidth=1.5)
ax1.plot(df.index, df['BB_upper'], label='上轨', color='red', linewidth=1, linestyle='--')
ax1.plot(df.index, df['BB_middle'], label='中轨', color='blue', linewidth=1)
ax1.plot(df.index, df['BB_lower'], label='下轨', color='green', linewidth=1, linestyle='--')
ax1.fill_between(df.index, df['BB_upper'], df['BB_lower'], alpha=0.1, color='blue')
ax1.set_ylabel('价格', fontsize=12)
ax1.set_title(f'{symbol} 布林带分析', fontsize=14)
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# 布林带宽度
ax2 = axes[1]
ax2.plot(df.index, df['BB_width'], color='purple', linewidth=1)
ax2.axhline(y=df['BB_width'].mean(), color='red', linestyle='--', alpha=0.5)
ax2.set_ylabel('带宽', fontsize=12)
ax2.set_title('布林带宽度 (Bandwidth)', fontsize=10)
ax2.grid(True, alpha=0.3)

# %B
ax3 = axes[2]
ax3.plot(df.index, df['BB_pctB'], color='orange', linewidth=1)
ax3.axhline(y=1, color='red', linestyle='--', linewidth=1, alpha=0.7)
ax3.axhline(y=0, color='green', linestyle='--', linewidth=1, alpha=0.7)
ax3.axhline(y=0.5, color='gray', linestyle='-', linewidth=0.5)
ax3.fill_between(df.index, 1, df['BB_pctB'].max(), color='red', alpha=0.1)
ax3.fill_between(df.index, df['BB_pctB'].min(), 0, color='green', alpha=0.1)
ax3.set_ylabel('%B', fontsize=12)
ax3.set_xlabel('日期', fontsize=12)
ax3.set_title('%B 指标', fontsize=10)
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. KDJ 指标分析

In [ ]:
# 计算 KDJ
df['K'], df['D'], df['J'] = kdj(df['high'], df['low'], df['close'], 9, 3, 3)

# 绘制 KDJ 图
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True,
                         gridspec_kw={'height_ratios': [2, 1]})

# 价格图
ax1 = axes[0]
ax1.plot(df.index, df['close'], label='收盘价', color='black', linewidth=1.5)
ax1.set_ylabel('价格', fontsize=12)
ax1.set_title(f'{symbol} KDJ 分析', fontsize=14)
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# KDJ 图
ax2 = axes[1]
ax2.plot(df.index, df['K'], label='K', color='blue', linewidth=1)
ax2.plot(df.index, df['D'], label='D', color='orange', linewidth=1)
ax2.plot(df.index, df['J'], label='J', color='purple', linewidth=1, alpha=0.7)

# 超买超卖线
ax2.axhline(y=80, color='red', linestyle='--', linewidth=1, alpha=0.7)
ax2.axhline(y=20, color='green', linestyle='--', linewidth=1, alpha=0.7)

ax2.fill_between(df.index, 80, 100, color='red', alpha=0.1)
ax2.fill_between(df.index, 0, 20, color='green', alpha=0.1)

ax2.set_ylabel('KDJ', fontsize=12)
ax2.set_xlabel('日期', fontsize=12)
ax2.legend(loc='upper left')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. ATR 波动率分析

In [ ]:
# 计算 ATR
df['ATR14'] = atr(df['high'], df['low'], df['close'], 14)
df['ATR_pct'] = df['ATR14'] / df['close'] * 100  # ATR 百分比

# 绘制 ATR 图
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True,
                         gridspec_kw={'height_ratios': [2, 1, 1]})

# 价格图
ax1 = axes[0]
ax1.plot(df.index, df['close'], label='收盘价', color='black', linewidth=1.5)
ax1.set_ylabel('价格', fontsize=12)
ax1.set_title(f'{symbol} ATR 波动率分析', fontsize=14)
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# ATR
ax2 = axes[1]
ax2.plot(df.index, df['ATR14'], color='blue', linewidth=1)
ax2.axhline(y=df['ATR14'].mean(), color='red', linestyle='--', alpha=0.5)
ax2.set_ylabel('ATR(14)', fontsize=12)
ax2.grid(True, alpha=0.3)

# ATR 百分比
ax3 = axes[2]
ax3.plot(df.index, df['ATR_pct'], color='purple', linewidth=1)
ax3.axhline(y=df['ATR_pct'].mean(), color='red', linestyle='--', alpha=0.5)
ax3.set_ylabel('ATR%', fontsize=12)
ax3.set_xlabel('日期', fontsize=12)
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"平均 ATR: {df['ATR14'].mean():.4f}")
print(f"平均 ATR%: {df['ATR_pct'].mean():.2f}%")

## 9. 信号生成与简单回测

In [ ]:
# 双均线交叉信号
df['signal'] = 0
df.loc[df['MA5'] > df['MA20'], 'signal'] = 1   # 金叉
df.loc[df['MA5'] < df['MA20'], 'signal'] = -1  # 死叉

# 计算信号变化点
df['signal_change'] = df['signal'].diff()

# 找出买卖点
buy_signals = df[df['signal_change'] == 2]   # 从 -1 变为 1
sell_signals = df[df['signal_change'] == -2]  # 从 1 变为 -1

print(f"买入信号: {len(buy_signals)} 次")
print(f"卖出信号: {len(sell_signals)} 次")

In [ ]:
# 绘制买卖信号图
fig, ax = plt.subplots(figsize=(14, 7))

ax.plot(df.index, df['close'], label='收盘价', color='black', linewidth=1.5)
ax.plot(df.index, df['MA5'], label='MA5', color='red', linewidth=1, alpha=0.8)
ax.plot(df.index, df['MA20'], label='MA20', color='blue', linewidth=1, alpha=0.8)

# 标记买卖点
ax.scatter(buy_signals.index, buy_signals['close'], 
           marker='^', color='red', s=100, label='买入', zorder=5)
ax.scatter(sell_signals.index, sell_signals['close'], 
           marker='v', color='green', s=100, label='卖出', zorder=5)

ax.set_ylabel('价格', fontsize=12)
ax.set_xlabel('日期', fontsize=12)
ax.set_title(f'{symbol} 双均线交叉信号 (MA5/MA20)', fontsize=14)
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 简单回测: 计算策略收益
df['returns'] = df['close'].pct_change()
df['strategy_returns'] = df['signal'].shift(1) * df['returns']

# 累计收益
df['cumulative_returns'] = (1 + df['returns']).cumprod()
df['cumulative_strategy'] = (1 + df['strategy_returns']).cumprod()

# 绘制累计收益对比
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(df.index, df['cumulative_returns'], label='持有策略', color='blue', linewidth=1.5)
ax.plot(df.index, df['cumulative_strategy'], label='双均线策略', color='red', linewidth=1.5)
ax.axhline(y=1, color='black', linestyle='--', alpha=0.5)

ax.set_ylabel('累计收益', fontsize=12)
ax.set_xlabel('日期', fontsize=12)
ax.set_title('策略收益对比', fontsize=14)
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 输出绩效
print("\n" + "="*50)
print("策略绩效对比")
print("="*50)
print(f"持有策略总收益: {(df['cumulative_returns'].iloc[-1] - 1)*100:.2f}%")
print(f"双均线策略总收益: {(df['cumulative_strategy'].iloc[-1] - 1)*100:.2f}%")

## 10. 多指标综合面板

In [ ]:
# 创建综合分析面板
fig, axes = plt.subplots(5, 1, figsize=(14, 16), sharex=True,
                         gridspec_kw={'height_ratios': [3, 1, 1, 1, 1]})

# 1. 价格与布林带
ax1 = axes[0]
ax1.plot(df.index, df['close'], label='收盘价', color='black', linewidth=1.5)
ax1.plot(df.index, df['MA5'], label='MA5', color='red', linewidth=1, alpha=0.7)
ax1.plot(df.index, df['MA20'], label='MA20', color='blue', linewidth=1, alpha=0.7)
ax1.fill_between(df.index, df['BB_upper'], df['BB_lower'], alpha=0.1, color='gray')
ax1.set_ylabel('价格', fontsize=10)
ax1.set_title(f'{symbol} 综合技术分析面板', fontsize=14)
ax1.legend(loc='upper left', fontsize=8)
ax1.grid(True, alpha=0.3)

# 2. MACD
ax2 = axes[1]
ax2.plot(df.index, df['DIF'], label='DIF', color='blue', linewidth=1)
ax2.plot(df.index, df['DEA'], label='DEA', color='orange', linewidth=1)
colors = ['red' if x >= 0 else 'green' for x in df['MACD']]
ax2.bar(df.index, df['MACD'], color=colors, alpha=0.6, width=1)
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax2.set_ylabel('MACD', fontsize=10)
ax2.legend(loc='upper left', fontsize=8)
ax2.grid(True, alpha=0.3)

# 3. RSI
ax3 = axes[2]
ax3.plot(df.index, df['RSI14'], label='RSI(14)', color='purple', linewidth=1)
ax3.axhline(y=70, color='red', linestyle='--', linewidth=1, alpha=0.7)
ax3.axhline(y=30, color='green', linestyle='--', linewidth=1, alpha=0.7)
ax3.set_ylim(0, 100)
ax3.set_ylabel('RSI', fontsize=10)
ax3.legend(loc='upper left', fontsize=8)
ax3.grid(True, alpha=0.3)

# 4. KDJ
ax4 = axes[3]
ax4.plot(df.index, df['K'], label='K', color='blue', linewidth=1)
ax4.plot(df.index, df['D'], label='D', color='orange', linewidth=1)
ax4.axhline(y=80, color='red', linestyle='--', linewidth=1, alpha=0.7)
ax4.axhline(y=20, color='green', linestyle='--', linewidth=1, alpha=0.7)
ax4.set_ylabel('KDJ', fontsize=10)
ax4.legend(loc='upper left', fontsize=8)
ax4.grid(True, alpha=0.3)

# 5. 成交量
ax5 = axes[4]
colors = ['red' if df['close'].iloc[i] >= df['open'].iloc[i] else 'green' 
          for i in range(len(df))]
ax5.bar(df.index, df['volume'], color=colors, alpha=0.7, width=1)
ax5.set_ylabel('成交量', fontsize=10)
ax5.set_xlabel('日期', fontsize=10)
ax5.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 11. 小结

本 Notebook 展示了常用技术指标的计算和可视化：

| 指标 | 用途 | 典型参数 |
|------|------|----------|
| MA/EMA | 趋势识别 | 5, 10, 20, 60 |
| MACD | 趋势强度和方向 | (12, 26, 9) |
| RSI | 超买超卖 | 14 |
| 布林带 | 波动区间 | (20, 2) |
| KDJ | 超买超卖 | (9, 3, 3) |
| ATR | 波动率 | 14 |

**下一步**: 前往 `03_strategy_prototype.ipynb` 学习策略原型开发。